In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# 예제 데이터 생성 (시계열 데이터 A와 B)
# A는 P1 ~ P10, B는 P1 ~ P15 (길이가 다른 시계열 데이터)
np.random.seed(42)
data_A = [np.random.rand(10) for _ in range(100)]  # 100개의 샘플
data_B = [np.random.rand(15) for _ in range(100)]  # 100개의 샘플

labels_A = np.random.randint(0, 2, size=100)  # 0 또는 1의 레이블
labels_B = np.random.randint(0, 2, size=100)

# 데이터와 레이블 통합
data = data_A + data_B
labels = np.concatenate([labels_A, labels_B])

# 데이터를 고정된 길이로 패딩 처리
max_length = 15  # 데이터에서 가장 긴 길이
padded_data = pad_sequences(data, padding='post', maxlen=max_length, dtype='float32')

# 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(padded_data, labels, test_size=0.2, random_state=42)

# 레이블을 0과 1로 인코딩 (이미 0과 1인 경우 생략 가능)
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

# LSTM 모델 정의
model = Sequential([
    Masking(mask_value=0.0, input_shape=(max_length, 1)),  # 패딩된 부분 무시
    LSTM(64, return_sequences=False),  # LSTM 레이어
    Dense(1, activation='sigmoid')  # 출력 레이어 (이진 분류)
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 데이터를 (samples, timesteps, features) 형태로 변환
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]

# 모델 학습
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_split=0.1)

# 모델 평가
y_pred = (model.predict(X_test) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.2f}")
